In [1]:
import pandas as pd
import faiss  
import numpy as np
import gc
import pickle
import os
from tqdm import tqdm
from itertools import chain
from os.path import join as path_join
from sklearn.decomposition import PCA

In [2]:
item_embeddings_path = "dumps/production/i2i/tmp_vacancies_embeddings"
vacancies_path = "dumps/production/i2i/tmp_vacancies"
test_data_path = "../data/processed/test_inference.parquet"
user_history_data_path = "dumps/user_history/history_data.parquet"

item_embeddings_path_2 = "dumps/production/i2i/name_area/embeddings.pickle"
vacancies_path_2 = "dumps/production/i2i/name_area/vacancies.parquet"

In [7]:
def load_embeddings(path):
    embeddings = pickle.load(open(path, 'rb'))
    embeddings = embeddings / np.linalg.norm(embeddings, axis = 1, keepdims=True)
    embeddings = np.float32(embeddings)

    pca = PCA(n_components=32)

    print("Fit PCA")
    pca.fit(embeddings[np.random.choice(len(embeddings), 50000, replace=False)])

    print("Transform PCA")
    embeddings = pca.transform(embeddings)

    print("Normalize PCA")
    embeddings = embeddings / np.linalg.norm(embeddings, axis = 1, keepdims=True)

    return embeddings

def load_item_2_idx(path):
    vacancies = pd.read_parquet(path)["vacancy_id"]
    return {a[1]: a[0] for a in enumerate(vacancies)}

def load_idx_2_item(path):
    vacancies = pd.read_parquet(path)["vacancy_id"]
    return {a[0]: a[1] for a in enumerate(vacancies)}

def load_test(test_data_path, user_history_data_path):
    test  = pd.read_parquet(test_data_path)
    user_history_data = pd.read_parquet(user_history_data_path)
    test = test.merge(user_history_data, "left", "user_id")
    return test

def load_vacancy_2_name(path):
    vacancies = pd.read_parquet(path)
    vacancy_ids = vacancies["vacancy_id"].to_list()
    names = vacancies["name"].to_list()
    
    return {x[0]: x[1] for x in zip(vacancy_ids, names)}

def load_vacancy_2_name_2(path):
    vacancies = pd.read_parquet(path)
    vacancy_ids = vacancies["vacancy_id"].to_list()
    names = vacancies["text"].to_list()
    
    return {x[0]: x[1] for x in zip(vacancy_ids, names)}

In [16]:
def get_user_embedding(vacancy_id, items, embeddings, item_2_idx):
    if isinstance(items, np.ndarray) or isinstance(items, list):
        history = list(chain(*items)) + list(vacancy_id)
    else:
        history = list(vacancy_id)

    history = filter(lambda x: x in item_2_idx, history)
    idxs = np.array(list(map(lambda vac: item_2_idx[vac], history)))

    if len(idxs) == 0:
        return np.zeros(32)
    
    user_embedding = embeddings[idxs].sum(axis=0)
    user_embedding = user_embedding / np.linalg.norm(user_embedding)

    return user_embedding

In [14]:
def get_hnsw(embeddings):
    index = faiss.IndexHNSWFlat(embeddings[0].shape[0], 32, faiss.METRIC_INNER_PRODUCT)
    # index = faiss.IndexFlatIP(embeddings[0].shape[0])
    index.add(embeddings)
    return index

In [11]:
embeddings = load_embeddings(item_embeddings_path_2)
item_2_idx = load_item_2_idx(vacancies_path_2)
idx_2_item = load_idx_2_item(vacancies_path_2)
test       = load_test(test_data_path, user_history_data_path)

Fit PCA
Transform PCA
Normalize PCA


In [12]:
vacancy_2_name = load_vacancy_2_name_2(vacancies_path_2)

In [17]:
user_embeddings = []

for row_idx, row in tqdm(test[["vacancy_id", "items"]].iterrows(), total=test.shape[0]):
    user_embeddings.append(get_user_embedding(row["vacancy_id"], row["items"], embeddings, item_2_idx))

user_embeddings = np.float32(np.array(user_embeddings))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46338/46338 [00:06<00:00, 7543.98it/s]


In [18]:
user_embeddings.shape, user_embeddings[0][0]

((46338, 32), 0.64761674)

In [19]:
hnsw = get_hnsw(embeddings)

In [20]:
D, I = hnsw.search(user_embeddings, 100)

In [23]:
test["prediction"] = list(map(
    lambda neighbours: list(map(
        lambda neighbour: idx_2_item[neighbour],
        filter(
            lambda neighbour: neighbour != -1,
            neighbours
        )
    )), 
    I
))

test["prediction"].head(40)

0     [v_1154508, v_2259119, v_1217003, v_2220490, v...
1     [v_1338343, v_2592979, v_337738, v_1737143, v_...
2     [v_170245, v_2477330, v_1742213, v_2163813, v_...
3     [v_1654097, v_927154, v_1757740, v_1466506, v_...
4     [v_2720047, v_1972973, v_387656, v_2416727, v_...
5     [v_610874, v_1368009, v_1387169, v_1138043, v_...
6     [v_1054977, v_2713977, v_1273431, v_343861, v_...
7     [v_2595366, v_1695596, v_1851768, v_2137777, v...
8     [v_1698479, v_2229346, v_2624185, v_2477864, v...
9     [v_2224208, v_2555629, v_1012743, v_2652157, v...
10    [v_2062023, v_2432149, v_19987, v_691447, v_14...
11    [v_2611142, v_1417845, v_2332498, v_2728754, v...
12    [v_583813, v_2363449, v_938678, v_1869347, v_8...
13    [v_2702905, v_2136207, v_797148, v_2489991, v_...
14    [v_2019636, v_692107, v_41464, v_2054227, v_26...
15    [v_1304345, v_1682733, v_490183, v_1467479, v_...
16    [v_1654097, v_927154, v_1757740, v_1466506, v_...
17    [v_33827, v_1434952, v_1196587, v_2658462,

In [22]:
def recall(row):
    predictions = row["prediction"]
    target = row["target_vacancy_id"]
    return int(target in set(predictions))

test[["prediction", "target_vacancy_id"]].apply(recall, axis=1).mean()

0.05656264836635159

In [123]:
for row_idx, row in test.iterrows():
    vacancy_id = row["vacancy_id"]
    items = row["items"]

    if isinstance(items, np.ndarray) or isinstance(items, list):
        history = list(chain(*items)) + list(vacancy_id)
    else:
        history = list(vacancy_id)

    if len(history) >= 5:
        continue
    
    print("=-=-=-=-=-==-=-=-=-=-=-=")

    print(row_idx)
    
    for vacancy in history:
        print(vacancy_2_name[vacancy])

    print("^^^^^^^^^^^^^^^^^^^^^^^^")

    print(vacancy_2_name[row["target_vacancy_id"]])
    
    print("^^^^^^^^^^^^^^^^^^^^^^^^")

    for vacancy in row["prediction"]:
        print(vacancy_2_name[vacancy])

    if row_idx > 60:
        break

=-=-=-=-=-==-=-=-=-=-=-=
3
Контролер ОТК
Менеджер по сопровождению продаж
Менеджер по сопровождению продаж
Техник-технолог в лабораторию
^^^^^^^^^^^^^^^^^^^^^^^^
Модель
^^^^^^^^^^^^^^^^^^^^^^^^
Менеджер по продажам
Менеджер по продажам
Менеджер по продажам
Менеджер по продажам
Менеджер по продажам
Менеджер по продажам
Менеджер по продажам
Менеджер по продажам
Менеджер по продажам
Менеджер по продажам
=-=-=-=-=-==-=-=-=-=-=-=
25
Главный бухгалтер
^^^^^^^^^^^^^^^^^^^^^^^^
Бухгалтер (удаленно)
^^^^^^^^^^^^^^^^^^^^^^^^
Главный бухгалтер
Главный бухгалтер
Главный бухгалтер
Главный бухгалтер
Главный бухгалтер
Главный бухгалтер
Главный бухгалтер
Главный бухгалтер
Главный бухгалтер
Главный бухгалтер
=-=-=-=-=-==-=-=-=-=-=-=
32
Оператор тепловых сетей (3 разряд), оперативно-диспетчерская служба
Инженер оперативно-диспетчерской службы
Ночной сборщик заказов (подработка ГПХ)
Сотрудник проката
^^^^^^^^^^^^^^^^^^^^^^^^
Помощник руководителя
^^^^^^^^^^^^^^^^^^^^^^^^
Инженер команды сопровождения ЕФС

In [112]:
debug_user = test.iloc[84]
debug_user

user_id                                     u_100201
session_id                                s_11981695
target_session_id                          s_2918385
vacancy_id                                [v_356656]
action_type                                      [2]
action_dt            [2023-11-10T13:18:23.851000000]
target_vacancy_id                          v_1230532
items                                            NaN
item_actions                                     NaN
Name: 84, dtype: object

In [113]:
debug_embedding = get_user_embedding(debug_user["vacancy_id"], debug_user["items"], embeddings, item_2_idx)
debug_embedding

array([ 0.5450181 , -0.39439932, -0.29821745,  0.02714841,  0.25700095,
       -0.19861138,  0.04498525, -0.0965538 ,  0.11373225, -0.11250065,
       -0.12182238, -0.13942687, -0.0999715 ,  0.19986533,  0.04747391,
        0.1814781 ,  0.10670567, -0.04522291, -0.1644038 , -0.08836821,
        0.06304052,  0.02392899, -0.09357398, -0.03642485,  0.2735493 ,
        0.10990983,  0.09712937, -0.12342686,  0.06223711,  0.00705061,
       -0.13061383,  0.02078187], dtype=float32)

In [114]:
embeddings[item_2_idx["v_356656"]]

array([ 0.5450181 , -0.39439932, -0.29821745,  0.02714841,  0.25700095,
       -0.19861138,  0.04498525, -0.0965538 ,  0.11373225, -0.11250065,
       -0.12182238, -0.13942687, -0.0999715 ,  0.19986533,  0.04747391,
        0.1814781 ,  0.10670567, -0.04522291, -0.1644038 , -0.08836821,
        0.06304052,  0.02392899, -0.09357398, -0.03642485,  0.2735493 ,
        0.10990983,  0.09712937, -0.12342686,  0.06223711,  0.00705061,
       -0.13061383,  0.02078187], dtype=float32)

In [115]:
D, I = hnsw.search(np.expand_dims(debug_embedding, axis=0), 10)
I

array([[1432313,  555767, 2290167, 2097993, 2360723, 1747826, 1270836,
        1805143, 1156497,  445814]])

In [116]:
for distance, debug_neighbour in zip(D[0], I[0]):
    print(distance, debug_neighbour, idx_2_item[debug_neighbour], vacancy_2_name[idx_2_item[debug_neighbour]])
    print(item_2_idx[idx_2_item[debug_neighbour]])

0.61555916 1432313 v_356656 Ассистент бизнес-аналитика, специалиста по внедрению (удаленно, стажер)
1432313
0.5660989 555767 v_1194379 Бизнес-ассистент консультантов
555767
0.5605482 2290167 v_1566319 Бизнес ассистент / бизнес аналитик (удаленно)
2290167
0.5570219 2097993 v_140345 Консультант по онлайн-курсам/ученик (удаленно)
2097993
0.5557258 2360723 v_2291664 Бизнес-аналитик / консультант(стажер)
2360723
0.5536852 1747826 v_1738703 Консультант по онлайн-курсам (удаленно)
1747826
0.5536852 1270836 v_69612 Консультант по онлайн-курсам (удаленно)
1270836
0.5529984 1805143 v_2037286 Консультант работодателей по найму персонала (удаленно)
1805143
0.5529984 1156497 v_411462 Консультант работодателей по найму персонала (удаленно)
1156497
0.5529984 445814 v_1272878 Консультант работодателей по найму персонала (удаленно)
445814


In [97]:
D, I = hnsw.search(np.expand_dims(embeddings[item_2_idx["v_356656"]], axis=0), 10)
I

array([[1432313, 2290167,  555767, 2360723,  974876, 2316198,   79197,
        2273414, 1172365, 1175787]])